## JSON 파일 파싱 

### JSON 파일에서 데이터를 뽑은 뒤에 우선 csv 파일로 전달, parquet로도 한번 변환
###  mysql에 넣어서 디스크 크기 비교,,(redshift 최적화 사례 찾아보기) 
###  프레임 일부 컬럼: 
    - 총 골드
    - 피해량 통계(총 피해량, 챔피언에게 준 총 피해량, 받은 총 피해량)
     서포터의 경우에는 쉴드량, 힐량이 없기 때문에 현재 측정이 어렵다는 문제점이 있으나, 일단 진행하기로 함
    - 정글 미니언 처치 수+미니언 처치 수
    - 챔피언 스탯 데이터
    - 레벨
    - 적 제어시간
    - 경험치 => 총 경험치 량으로 진행
### 이벤트 
    - 전체 이벤트 데이터를 넣는 것을 목표로 함

In [7]:
import os
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [8]:
from operator import itemgetter

## 챔피언 정보 가져오기 (match data)


In [9]:
championList = list()

In [10]:
with open('./data/match/KR_7289807623.json') as f:
    matchData = json.load(f)

In [11]:
matchData

{'metadata': {'dataVersion': '2',
  'matchId': 'KR_7289807623',
  'participants': ['IuhfaScGFL-BHkNH-782z09fRpwCdwSdic7mF0ZO11sszOkgW0SG4anDRNL-G7FrS5Qiu-Z98nlOgg',
   '8I9iUjoU20bR3cGtLbuo2_wfTZBp4fFD4qdMOqhSrsHgbCFd5YYAAblgylSQZEFNfl1Ejc27JvpFtw',
   'zsM30eDlAlx24wJtlOQ7zzuV-goWDCTBpc_Kb153jYuj2s7iIvyvPIWhkI6v33JHwESnPfVVtcD5Hw',
   'sZCAKc9RebxOAVNkP2vFzVYO9m53haGY5ybgsQvMYEvgmUeV6oNaoG793x1UTFpJHADpmP9tBul0vQ',
   '7vW88m56F3uaRVYZNXSxG5FuuReA2C4HkDE-Sqh91LShAciz-Zwe0uqVUi24-wx0stAnuH_Pre5hIA',
   'JtsvF4Thr4wRBjyYz7nebQFq0z5R2TaYmF6hT2XR-lpD9m7LBKDjJUTsfnnI-4qeDWF4Mq_DamR33g',
   'rjWl6lp3AZsQpV60Oj5xkclQy8WnriiCxd-6PP0k4yoHGSy685g4F4wC-pewoH75p88BJbww1gtvrQ',
   'M_nVzt_ZizaGXIsSD6Viv6EmjTHpDnFCklkToIve8Q8qmZD8Zgobr3Z7CoLjKM4nuxUcn-KE867euQ',
   'WaWpa0QCYOKXM-JpT-JTDNDAr2KzuetF20_NNuQav8eBncwmifz4GA5cyk2HaTQaD8-hpdp4gBJZoQ',
   'n0eLE2rxElGLbWkenl71VEijYS4IR9J6p5vD50CUqcEwiNanOw2B9oTArI0JKErs9ttwMG-Xxm_sDg']},
 'info': {'endOfGameResult': 'GameComplete',
  'gameCreation': 17267

In [12]:
matchData['metadata']

{'dataVersion': '2',
 'matchId': 'KR_7289807623',
 'participants': ['IuhfaScGFL-BHkNH-782z09fRpwCdwSdic7mF0ZO11sszOkgW0SG4anDRNL-G7FrS5Qiu-Z98nlOgg',
  '8I9iUjoU20bR3cGtLbuo2_wfTZBp4fFD4qdMOqhSrsHgbCFd5YYAAblgylSQZEFNfl1Ejc27JvpFtw',
  'zsM30eDlAlx24wJtlOQ7zzuV-goWDCTBpc_Kb153jYuj2s7iIvyvPIWhkI6v33JHwESnPfVVtcD5Hw',
  'sZCAKc9RebxOAVNkP2vFzVYO9m53haGY5ybgsQvMYEvgmUeV6oNaoG793x1UTFpJHADpmP9tBul0vQ',
  '7vW88m56F3uaRVYZNXSxG5FuuReA2C4HkDE-Sqh91LShAciz-Zwe0uqVUi24-wx0stAnuH_Pre5hIA',
  'JtsvF4Thr4wRBjyYz7nebQFq0z5R2TaYmF6hT2XR-lpD9m7LBKDjJUTsfnnI-4qeDWF4Mq_DamR33g',
  'rjWl6lp3AZsQpV60Oj5xkclQy8WnriiCxd-6PP0k4yoHGSy685g4F4wC-pewoH75p88BJbww1gtvrQ',
  'M_nVzt_ZizaGXIsSD6Viv6EmjTHpDnFCklkToIve8Q8qmZD8Zgobr3Z7CoLjKM4nuxUcn-KE867euQ',
  'WaWpa0QCYOKXM-JpT-JTDNDAr2KzuetF20_NNuQav8eBncwmifz4GA5cyk2HaTQaD8-hpdp4gBJZoQ',
  'n0eLE2rxElGLbWkenl71VEijYS4IR9J6p5vD50CUqcEwiNanOw2B9oTArI0JKErs9ttwMG-Xxm_sDg']}

In [13]:
keysToGet = ['matchId', 'championId','championName','participantId']

In [15]:
championList = list()
for championData in matchData['info']['participants']:
    values = itemgetter(*['participantId', 'championId','championName'])(championData)
    championList.append((matchData['metadata']['matchId'],)+ values  )


In [16]:
championList

[('KR_7289807623', 1, 58, 'Renekton'),
 ('KR_7289807623', 2, 11, 'MasterYi'),
 ('KR_7289807623', 3, 901, 'Smolder'),
 ('KR_7289807623', 4, 8, 'Vladimir'),
 ('KR_7289807623', 5, 432, 'Bard'),
 ('KR_7289807623', 6, 62, 'MonkeyKing'),
 ('KR_7289807623', 7, 234, 'Viego'),
 ('KR_7289807623', 8, 136, 'AurelionSol'),
 ('KR_7289807623', 9, 51, 'Caitlyn'),
 ('KR_7289807623', 10, 201, 'Braum')]

In [17]:
championDf = pd.DataFrame(championList, columns = ['matchId','participantId','championId', 'championName'])

In [18]:
championDf.head()

,matchId,participantId,championId,championName
0,KR_7289807623,1,58,Renekton
1,KR_7289807623,2,11,MasterYi
2,KR_7289807623,3,901,Smolder
3,KR_7289807623,4,8,Vladimir
4,KR_7289807623,5,432,Bard


json파일 파싱 
(지금은 테스트를 위해 파일 하나를 가지고 넣음)

In [19]:
# data dir
json_dir = './data/match_timeline/'
json_files = os.listdir(json_dir)

len(json_files)

2

In [20]:
with open('./data/match_timeline/match_timeline/KR_7289807623.json') as f:
    data = json.load(f)

## Champion 기준으로 한번 봅시다 

In [113]:
championTable = pd.DataFrame(columns=['assistingParticipantIds', 'bounty','killStreakLength',\
                                   'killerId','position','shutdownBounty','timestamp',\
                                    'victimDamageDealt','victimDamageReceived','victimId','type'])

In [114]:
champion_dict = dict()


### 테이블1 : 챔피언 stat part
데이터 파싱 방법: 'frame'에 있는 정보를 가져오면 됨. 여기서 어떻게 가져오는가?   

데이터 처리 코드 작성 후, 창길님 코드에 맞추어서 다시 함수로 리팩토링 

In [115]:
participantNumber = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
frameList = data['info']['frames']
testDf = list()
for i in range(0, len(data['info']['frames']),1):
    for pN in range(1, 9, 1):
        testDf.append({**frameList[i]['participantFrames'][str(pN)]['championStats'],\
                       'participantId' : frameList[i]['participantFrames'][str(pN)]['participantId'],\
                       'currentGold': frameList[i]['participantFrames'][str(pN)]['currentGold'],\
                        **frameList[i]['participantFrames'][str(pN)]['damageStats'],\
                        'goldPerSecond':frameList[i]['participantFrames'][str(pN)]['goldPerSecond'],\
                        'jungleMinionKilled': frameList[i]['participantFrames'][str(pN)]['jungleMinionsKilled'],\
                        'level': frameList[i]['participantFrames'][str(pN)]['level'],\
                        'minionKilled': frameList[i]['participantFrames'][str(pN)]['minionsKilled'],\
                        'timeEnemySpendControlled':frameList[i]['participantFrames'][str(pN)]['timeEnemySpentControlled'],\
                        'totalGold':frameList[i]['participantFrames'][str(pN)]['totalGold'],\
                        'xp':frameList[i]['participantFrames'][str(pN)]['xp']}
                        
                        )


In [119]:
test = pd.DataFrame(testDf)
participantstat = pd.merge(championDf[['matchId','participantId','championId']], test, on = 'participantId')

In [126]:
participantstat

,matchId,participantId,championId,abilityHaste,abilityPower,armor,armorPen,armorPenPercent,attackDamage,attackSpeed,...,trueDamageDone,trueDamageDoneToChampions,trueDamageTaken,goldPerSecond,jungleMinionKilled,level,minionKilled,timeEnemySpendControlled,totalGold,xp
0,KR_7289807623,1,58,0,0,35,0,0,25,100,...,0,0,0,0,0,1,0,0,500,0
1,KR_7289807623,1,58,0,0,35,0,0,79,103,...,0,0,0,0,0,1,0,0,500,0
2,KR_7289807623,1,58,0,0,35,0,0,79,103,...,0,0,0,0,0,1,1,0,542,61
3,KR_7289807623,1,58,0,0,38,0,0,92,106,...,0,0,90,0,0,2,5,4074,1159,636
4,KR_7289807623,1,58,0,0,46,0,0,101,110,...,0,0,90,0,0,4,18,4074,1552,1499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,KR_7289807623,8,136,0,236,45,0,0,96,118,...,4359,289,190,0,0,14,188,405492,9458,12001
228,KR_7289807623,8,136,0,264,70,0,0,96,118,...,4859,289,190,0,0,14,194,452305,9763,12257
229,KR_7289807623,8,136,0,241,75,0,0,100,119,...,5331,289,535,0,0,15,201,491018,11032,13676
230,KR_7289807623,8,136,0,356,75,0,0,100,119,...,6156,706,802,0,0,15,208,560152,11349,14104


## 와드 제거 

In [52]:
wardRemoveList = list()

In [53]:
#여기서 event 데이터를 어떻게 파싱했더라?
#이 데이터에서 type은 따로 전처리 해야함(여기서 창길님 코드 참고)

for i in range(0, len(data['info']['frames']),1):
     element = data['info']['frames'][i]['events']
     #print(element)
     for e in element:
          if e['type'] == "WARD_KILL":
               wardRemoveList.append(e)
     #필요한 정보: timestamp, type, wardType
     #join을 걸어야 하는 정보: killerId와 match정보를 Join걸어야 함(을 'participantId'로 해서 빼면 됨)
     

In [55]:
wardRemoveDf = pd.DataFrame(wardRemoveList)

In [58]:
wardRemoveDf = wardRemoveDf[['killerId', 'timestamp', 'wardType']]

In [59]:
wardRemoveDf

,killerId,timestamp,wardType
0,3,270777,YELLOW_TRINKET
1,8,288715,CONTROL_WARD
2,7,520776,YELLOW_TRINKET
3,10,559829,SIGHT_WARD
4,5,559929,CONTROL_WARD
...,...,...,...
63,9,1528948,UNDEFINED
64,1,1565198,SIGHT_WARD
65,10,1584456,CONTROL_WARD
66,5,1608754,BLUE_TRINKET


In [127]:

wardRemove= pd.merge(championDf[['matchId','participantId']], test, on = 'participantId')
wardRemove

,matchId,participantId,abilityHaste,abilityPower,armor,armorPen,armorPenPercent,attackDamage,attackSpeed,bonusArmorPenPercent,...,trueDamageDone,trueDamageDoneToChampions,trueDamageTaken,goldPerSecond,jungleMinionKilled,level,minionKilled,timeEnemySpendControlled,totalGold,xp
0,KR_7289807623,1,0,0,35,0,0,25,100,0,...,0,0,0,0,0,1,0,0,500,0
1,KR_7289807623,1,0,0,35,0,0,79,103,0,...,0,0,0,0,0,1,0,0,500,0
2,KR_7289807623,1,0,0,35,0,0,79,103,0,...,0,0,0,0,0,1,1,0,542,61
3,KR_7289807623,1,0,0,38,0,0,92,106,0,...,0,0,90,0,0,2,5,4074,1159,636
4,KR_7289807623,1,0,0,46,0,0,101,110,0,...,0,0,90,0,0,4,18,4074,1552,1499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,KR_7289807623,8,0,236,45,0,0,96,118,0,...,4359,289,190,0,0,14,188,405492,9458,12001
228,KR_7289807623,8,0,264,70,0,0,96,118,0,...,4859,289,190,0,0,14,194,452305,9763,12257
229,KR_7289807623,8,0,241,75,0,0,100,119,0,...,5331,289,535,0,0,15,201,491018,11032,13676
230,KR_7289807623,8,0,356,75,0,0,100,119,0,...,6156,706,802,0,0,15,208,560152,11349,14104


## 스킬 레벨업 

In [28]:
skilLevelUp = list()

In [29]:
for i in range(0, len(data['info']['frames']),1):
     element = data['info']['frames'][i]['events']
     #print(element)
     for e in element:
          if e['type'] == "SKILL_LEVEL_UP":
               skilLevelUp.append(e)

In [30]:
skillLevelUpDf = pd.DataFrame(skilLevelUp)

In [31]:
skillLevelUpDf = skillLevelUpDf[['levelUpType', 'participantId', 'skillSlot', 'timestamp']]

In [32]:
skilLevelUp = pd.merge(championDf[['matchId','participantId']], skillLevelUpDf, on = 'participantId')
skilLevelUp

,matchId,participantId,levelUpType,skillSlot,timestamp
0,KR_7289807623,1,NORMAL,1,100047
1,KR_7289807623,1,NORMAL,3,147772
2,KR_7289807623,1,NORMAL,2,185375
3,KR_7289807623,1,NORMAL,1,218946
4,KR_7289807623,1,NORMAL,1,262295
...,...,...,...,...,...
141,KR_7289807623,10,NORMAL,1,879655
142,KR_7289807623,10,NORMAL,2,1017448
143,KR_7289807623,10,NORMAL,4,1145265
144,KR_7289807623,10,NORMAL,2,1281342


## 레벨 업

In [34]:
levelUp = list()

In [35]:
for i in range(0, len(data['info']['frames']),1):
     element = data['info']['frames'][i]['events']
     #print(element)
     for e in element:
          if e['type'] == "LEVEL_UP":
               levelUp.append(e)

In [36]:
levelUpDf = pd.DataFrame(levelUp)

In [25]:
levelUpDf = levelUpDf[['level', 'participantId', 'timestamp']]

In [26]:
levelUpDf

,level,participantId,timestamp
0,2,7,104189
1,2,2,105825
2,2,3,123993
3,2,8,127066
4,2,6,131140
...,...,...,...
131,15,4,1611778
132,13,5,1611778
133,17,1,1614571
134,15,2,1621019


## 아이템 판매 

In [37]:
itemSold = list()

In [39]:
for i in range(0, len(data['info']['frames']),1):
     element = data['info']['frames'][i]['events']
     #print(element)
     for e in element:
          if e['type'] == "ITEM_SOLD":
               itemSold.append(e)

In [40]:
itemSoldDf = pd.DataFrame(itemSold)

In [42]:
itemSoldDf = itemSoldDf[['itemId', 'participantId', 'timestamp']]

In [43]:
itemSoldDf = pd.merge(championDf[['matchId','participantId']], itemSoldDf, on = 'participantId')
itemSoldDf

,matchId,participantId,itemId,timestamp
0,KR_7289807623,1,1054,884074
1,KR_7289807623,1,2031,884408
2,KR_7289807623,1,2031,887280
3,KR_7289807623,1,1054,1229830
4,KR_7289807623,1,1054,1252646
5,KR_7289807623,4,2055,716797
6,KR_7289807623,4,2031,1230331
7,KR_7289807623,7,2055,1464859
8,KR_7289807623,8,1082,1460518


## 게임 종료 

In [46]:
gameOver = list()

In [48]:
for i in range(0, len(data['info']['frames']),1):
     element = data['info']['frames'][i]['events']
     #print(element)
     for e in element:
          if e['type'] == "GAME_END":
               gameOver.append(e)
               

In [50]:
gameOver

[{'gameId': 7289807623,
  'realTimestamp': 1726771789125,
  'timestamp': 1672079,
  'type': 'GAME_END',
  'winningTeam': 100}]

In [53]:
gameOverDf = pd.DataFrame(gameOver)

In [54]:
gameOverDf['matchId'] = matchData['metadata']['matchId']

In [55]:
gameOverDf

,gameId,realTimestamp,timestamp,type,winningTeam,matchId
0,7289807623,1726771789125,1672079,GAME_END,100,KR_7289807623


In [57]:
gameOverDf = gameOverDf[['matchId', 'gameId', 'realTimestamp', 'timestamp', 'winningTeam']]

In [58]:
gameOverDf

,matchId,gameId,realTimestamp,timestamp,winningTeam
0,KR_7289807623,7289807623,1726771789125,1672079,100
